<a href="https://colab.research.google.com/github/plz77/Data-Quality-Validation-System/blob/main/Sistem_Validasi_Kualitas_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import unicodedata
from transformers import pipeline

"""
--------------------------------------------------------------------------------
Modul Pelatihan: Kualitas Data (Data Quality) untuk AI
--------------------------------------------------------------------------------
Halo! Proyek ini berfokus pada "Penyaringan Data". Sebelum AI belajar, kita
harus memastikan data yang ia baca bersih. Data yang kotor (sampah) akan
membuat AI menjadi bodoh atau tidak akurat.

Tujuan Proyek:
Membangun sistem otomatis yang memvalidasi apakah sebuah teks layak masuk ke
dalam dataset pelatihan (training set) atau harus dibuang.

Istilah Penting:
1. Noise Cleaning: Menghapus karakter yang tidak perlu (seperti emoji berlebih
   atau simbol aneh).
2. Heuristic Filter: Aturan logika (seperti panjang kata) untuk membuang teks
   yang tidak bermakna.
3. Data Integrity: Memastikan data unik dan memiliki struktur yang benar.
--------------------------------------------------------------------------------
"""

class DataQualityGuard:
    def __init__(self):
        print("Sistem Data Quality Guard diaktifkan...")
        # Memuat model sentimen standar sebagai bagian dari pipeline validasi
        self.sentiment_model = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

    def clean_text(self, text):
        """Membersihkan teks dari karakter yang tidak perlu."""
        # Menghapus spasi berlebih
        text = " ".join(text.split())
        # Normalisasi karakter (menghapus aksen)
        text = "".join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
        return text

    def is_high_quality(self, text):
        """Memeriksa apakah teks memenuhi standar kualitas (Akurasi Tinggi)."""

        # 1. Cek Panjang Teks (Minimal 10 karakter agar bermakna)
        if len(text) < 10:
            return False, "Ditolak: Teks terlalu pendek (Informasi tidak cukup)."

        # 2. Cek Karakter Berulang (Mendeteksi spam/spamming karakter)
        if re.search(r'(.)\1{3,}', text):
            return False, "Ditolak: Terdeteksi karakter berulang (Potensi spam)."

        # 3. Cek Rasio Kata (Memastikan teks memiliki struktur kalimat)
        words = text.split()
        if len(words) < 3:
            return False, "Ditolak: Bukan merupakan kalimat lengkap."

        # 4. Cek Karakter Spesial Berlebihan
        special_chars = re.findall(r'[^a-zA-Z0-9\s]', text)
        if len(special_chars) / len(text) > 0.3:
            return False, "Ditolak: Terlalu banyak simbol/karakter spesial."

        return True, "Lolos: Data berkualitas tinggi."

    def process_dataset(self, raw_data):
        """Memproses sekumpulan data dan memisahkan data bersih vs kotor."""
        clean_batch = []
        rejected_log = []

        print(f"\nMulai memproses {len(raw_data)} data mentah...\n")

        for item in raw_data:
            original_text = item
            cleaned = self.clean_text(original_text)
            is_valid, reason = self.is_high_quality(cleaned)

            if is_valid:
                # Jika lolos validasi, hitung sentimen
                sentiment = self.sentiment_model(cleaned)[0]
                clean_batch.append({
                    "text": cleaned,
                    "sentiment": sentiment['label'],
                    "confidence": f"{sentiment['score']:.2%}"
                })
            else:
                rejected_log.append({"text": original_text, "reason": reason})

        return clean_batch, rejected_log

# --- Simulasi Penggunaan ---
if __name__ == "__main__":
    guard = DataQualityGuard()

    # Dataset mentah yang bercampur antara data bagus dan sampah
    raw_dataset = [
        "The teacher explained the concept of AI very clearly today.",
        "pashduashduashduashduashduashduashdua", # Sampah
        "I hate this class!!!!!! @#$%^&*",        # Terlalu banyak simbol
        "Great lesson.",                           # Terlalu pendek
        "Python is the most popular language for data science in 2025.",
        "Learning is funnnnnnnnnn",                # Karakter berulang
        "The curriculum is well-structured and easy to follow."
    ]

    valid_data, rejected_data = guard.process_dataset(raw_dataset)

    print("--- DATA YANG LOLOS (SIAP TRAINING) ---")
    for data in valid_data:
        print(f"[PASSED] {data['text']} | Sentimen: {data['sentiment']} ({data['confidence']})")

    print("\n--- DATA YANG DITOLAK (REJECTED) ---")
    for data in rejected_data:
        print(f"[REJECTED] {data['text']} | Alasan: {data['reason']}")

    total = len(raw_dataset)
    print(f"\nRingkasan Kualitas: {len(valid_data)}/{total} data bersih ({(len(valid_data)/total):.0%})")